In [1]:
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [6]:
def escalar_imagen(image, max_width):
    original_height, original_width = image.shape
    ratio = max_width / original_width
    height = int(original_height * ratio)
    resize=cv2.resize(image, (max_width, height))
    return resize

def mostrar_imagen(image, label, max_width):
    imagen_resized = escalar_imagen(image, max_width)
    imagen_tk = ImageTk.PhotoImage(image=Image.fromarray(imagen_resized))
    label.config(image=imagen_tk)
    label.image = imagen_tk

def actualizar_filtro(valor):
    fil, col = img.shape

    img_padded = np.pad(img,((0,fil),(0,col)),'constant',constant_values=((0, 0),(0,0)))

    f, c = np.ogrid[0:2*fil,0:2*col]

    H = ((f-fil)**2 + (c-col)**2) <=  (int(valor)*fil/100)**2
    H_pb_ideal = np.float32(H)

    img_fft = np.fft.fft2(img_padded)
    img_fft_shift = np.fft.fftshift(img_fft)

    G_fft = img_fft_shift * H_pb_ideal

    G_fft_ishift = np.fft.ifftshift(G_fft)
    g = np.fft.ifft2(G_fft_ishift)
    g_real = np.real(g)[0:fil, 0:col]
    
    mostrar_imagen(H_pb_ideal/np.max(H_pb_ideal)*255, label_matriz, 300)
    mostrar_imagen(g_real, label_g_real, 300)

In [8]:
image_path = "imagen2.png"  # Ruta de la imagen a mostrar
img = cv2.imread(image_path, 0)

ventana = tk.Tk()
ventana.title("Mostrar Imagen y Matriz con Tkinter")

label_imagen = tk.Label(ventana)
mostrar_imagen(img, label_imagen, 300)
label_imagen.grid(row=0, column=0, padx=10, pady=10)

label_matriz = tk.Label(ventana)
label_matriz.grid(row=0, column=1, padx=10, pady=10)

label_g_real = tk.Label(ventana)
label_g_real.grid(row=0, column=2, padx=10, pady=10)

slider_n = tk.Scale(ventana, from_=1, to=50, orient=tk.HORIZONTAL,
                    label="Valor de n", command=actualizar_filtro)
slider_n.set(2)
slider_n.grid(row=1, column=0, columnspan=3, pady=10)

ventana.mainloop()